In [13]:
import numpy as np
import cv2
import threading
import time, os
import matplotlib.pyplot as plt

In [14]:
BGR_COLOR = {
    'red': (0,0,255),
    'green': (127,255,0),
    'blue': (255,127,0),
    'yellow': (0,127,255),
    'black': (0,0,0),
    'white': (255,255,255)
}

In [15]:
class WebcamVideoStream:
    
    def __init__(self, src=0, api=None):
        self.stream = cv2.VideoCapture(src, api) if api else cv2.VideoCapture(src)
        
        self.stream.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))
        self.stream.set(cv2.CAP_PROP_FRAME_WIDTH, 1024)
        self.stream.set(cv2.CAP_PROP_FRAME_HEIGHT, 768)
        self.stream.set(cv2.CAP_PROP_FPS, 20)
        self.stream.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))
        #self.stream.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('H', '2', '6', '4'))
                
#         print("W:H - %s:%s" % (self.stream.get(cv2.CAP_PROP_FRAME_WIDTH), self.stream.get(cv2.CAP_PROP_FRAME_HEIGHT)))
#         print("FPS: %s" % self.stream.get(cv2.CAP_PROP_FPS))
#         print(self.stream.get(cv2.CAP_PROP_FOURCC))
#         print(self.stream.get(cv2.CAP_PROP_BACKEND))
        
        self.frame = None
        self.stopped = False
        self.frame_counter = []
        
    def start(self):
        th = threading.Thread(target=self.update, args=())
        th.start()
        return th

    def stop(self):
        self.stopped = True
        self.stream.release()
        
    def update(self):
        while not self.stopped:
            (self.grabbed, self.frame) = self.stream.read()
            self.frame_counter.append(time.time())
            
    def read(self):
        return self.frame

    def get_frame_diffs(self):
        return 1.0/np.diff(np.array(self.frame_counter))
        
    def get_avg_fps(self):
        return self.get_frame_diffs().mean()

In [16]:
class VideoWriter:
    
    def __init__(self, vs):
        self.fps = 20.0
        self.vs = vs
        self.stopped = False
        self.frame_counter = []
        
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        self.out = cv2.VideoWriter('test_video.avi', fourcc, self.fps, (int(vs.stream.get(3)), int(vs.stream.get(4))))
        
    def start(self):
        self.t_start = time.time()
        th = threading.Thread(target=self.update, args=())
        th.start()
        return th
    
    def stop(self):
        self.stopped = True
        self.out.release()
        
    def update(self):
        while not self.stopped:
            if time.time() < self.t_start + 1./self.fps * float(len(self.frame_counter)):
                continue

            frame = self.vs.read()
            if frame is not None:
                self.out.write(frame)
                self.frame_counter.append(time.time())
                if len(self.frame_counter) == 1:
                    self.t_start = self.frame_counter[0]
                
    def get_frame_diffs(self):
        return 1.0/np.diff(np.array(self.frame_counter))
        
    def get_avg_fps(self):
        return self.get_frame_diffs().mean()                

In [17]:
class PositionTracker:
    
    def __init__(self, video_streamer, background_file='background.png'):
        self.video_streamer = video_streamer
        self.background = cv2.imread(background_file, 1)
        self.fps = 20.0
        self.x, self.y = None, None
        self.contour = []
        self.stopped = False
        self.frame_counter = []
        
        self.mask = np.zeros(shape=self.background.shape, dtype="uint8")
        cv2.circle(self.mask, (512, 384), 350, BGR_COLOR['white'], -1)
        
    def start(self):
        self.t_start = time.time()
        th = threading.Thread(target=self.update, args=())
        th.start()
        return th
    
    def stop(self):
        self.stopped = True
        
    def update(self):
        while not self.stopped:
            if time.time() < self.t_start + 1./self.fps * float(len(self.frame_counter)):
                continue

            frame = self.video_streamer.read()
            if frame is not None:
                self.detect_position(frame)
                self.frame_counter.append(time.time())
                if len(self.frame_counter) == 1:
                    self.t_start = self.frame_counter[0]
                
    def get_frame_diffs(self):
        return 1.0/np.diff(np.array(self.frame_counter))
        
    def get_avg_fps(self):
        return self.get_frame_diffs().mean()          
       
    def detect_position(self, frame):
        masked_frame = cv2.bitwise_and(src1=frame, src2=self.mask)

        # Substracts background from current frame
        subject = cv2.subtract(masked_frame, self.background)

        # Converts subject to grey scale
        subject_gray = cv2.cvtColor(subject, cv2.COLOR_BGR2GRAY)

        # Applies blur and thresholding to the subject
        kernel_size = (25,25)
        frame_blur = cv2.GaussianBlur(subject_gray, kernel_size, 0)
        _, thresh = cv2.threshold(frame_blur, 40, 255, cv2.THRESH_BINARY)

        # Finds contours and selects the contour with the largest area
        contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

        if len(contours) == 0:
            return
        contour = contours[np.argmax(list(map(cv2.contourArea, contours)))]
        M = cv2.moments(contour)
        if (M['m00'] == 0):
            return

        self.x = int(M['m10'] / M['m00'])
        self.y = int(M['m01'] / M['m00'])
        self.contour = contour

In [19]:
# TODO update FPS counters

In [20]:
# check camera output formats with FFMPEG
# https://stackoverflow.com/questions/15301608/how-to-query-a-webcams-output-formats
#
# ffmpeg -list_devices true -f dshow -i dummy
# ffmpeg -list_options true -f dshow -i video="HD USB Camera"

In [22]:
# https://stackoverflow.com/questions/39308664/opencv-cant-set-mjpg-compression-for-usb-camera
#vs = WebcamVideoStream(src=0, api=cv2.CAP_DSHOW)  # for IR usb camera api=cv2.CAP_DSHOW
vs = WebcamVideoStream(src=0)
t1 = vs.start()

vw = VideoWriter(vs)
t2 = vw.start()

pt = PositionTracker(vs, background_file=os.path.join('assets', 'background.png'))
t3 = pt.start()

#wc = WindowController(vs, pt)
#t4 = wc.start()


frame_counter = []
try:
    while True:
        frame_counter.append(time.time())
        
        frame = vs.read()
        if frame is not None:
            if pt.x is not None:
                cv2.circle(frame, (pt.x, pt.y), 3, BGR_COLOR['yellow'], -1)
            cv2.imshow("Experiment", frame)

        k = cv2.waitKey(0)
        if k == ord('q'):
            break

finally:
    cv2.destroyAllWindows()
    print('Windows destroyed')   
    vs.stop()
    print('Streamer stopped')
    pt.stop()
    print('Position stopped')
    time.sleep(2)
    vw.stop()
    print('Writer stopped')
    for t in [t1, t2, t3]:
        t.join()
        
    print('Threads joined')

Windows destroyed
Streamer stopped
Position stopped
Writer stopped
Threads joined


In [ ]:
vs.get_avg_fps(), vw.get_avg_fps(), pt.get_avg_fps()

In [ ]:
_ = plt.hist(pt.get_frame_diffs(), 50)

In [ ]:
plt.plot(pt.get_frame_diffs())